# Rendezvous with Zarya

[ NB: ALL OF THI

Now to tray and track a satellite from the ground.  For this, we're going to shoot for Zarya (well, actually, the entire ISS, not just its first component).  This is an easy thing to see by eye, and is also easily tracked in gpredict or other tools.

First up, let's copypasta our ECI stuff from `simple_eci_frames.ipynb`, because libraries are for people who know what they're doing already.

In [1]:
%pylab inline
import numpy as np # Going to use numpy for lots of matrix stuff, sorrynotsorry

class GMST:
    T0_utc = 946728000 # 2000-01-01T12h00Z is the zero point for T below
    GMST0 = 18.697374558
    SID_HRS = 24.06570982441908 # Clock hours in a sideral day
    
    
    @classmethod
    def from_unix(cls, t_utc):
        """Compute GMST (in radians) based on unix time
        
        Based on a formula from http://aa.usno.navy.mil/faq/docs/GAST.php
        
        This formula is reportedly good to 0.1sec/century, which is plenty
        good for SGP4 work, where we're willing to conflate UTC and UT1, which
        is a margin of a millenium(!) of drift due to this formula.
        
        NB: I'm pretty sure we get all the significant figures in the constants
        here, but it's not super clear, so the drift may be higher than
        reported.  For this application, I can't be bothered to check up on this,
        but if you need super high precision, this is a bit of an XXX comment.
        """
        
        d = (t_utc - GMST.T0_utc)/86400.0
        gmst_h = (GMST.GMST0 + GMST.SID_HRS*d) % 24 # GMST given in 24.00 hour periods
        gmst_rad = gmst_h / 24.00*2*np.pi # 24.00 hr, so not the same as WGS84.omega
        return gmst_rad
    
    
class WGS84:
    # A container for random WGS84 constants; NB: this duplicates stuff in sgp4, probably can be removed
    f = 1/298.257223560 # flattening of the spheroid, no units
    a = 6378.137 # equatorial radius, km
    omega_e = 366.25/365.25 # ratio of sidereal day to solar day
    omega = 7.292115e-5 # earth rotation rate, radians/second. Note! this is 2pi/_sidereal_ day
                        # Alternately, omega = 2*pi/86400*omega_e
    

class ECIPoint:
    "Represents an (x,y,z) point with velocity in the Earth-Centered Inertial coordinate system"
    def __init__(self, pos, vel):
        self.pos = np.array(pos)
        self.vel = np.array(vel)
        
    def speed(self):
        # Get the absolute speed of this object
        return np.linalg.norm(self.vel)
    
    def range_to(self, eci2):
        # Get the range from this point to the second point
        return eci2.pos - self.pos
    
    def range_vel(self, eci2):
        return eci2.vel - self.vel
    
    def alt(self, t):
        return np.linalg.norm(self.pos) - WGS84.a
    
    def lat(self,t ):
        # Get the latitude under this point, in radians
        return 0.00
        
    def __repr__(self):
        return "(%0.3f,%0.3f,%0.3f) km @ (%0.3f,%0.3f,%0.3f) km/s" % (self.pos[0], self.pos[1], self.pos[2],
                                                                      self.vel[0], self.vel[1], self.vel[2])
    
class ECIEarthPoints:
    "Given a lat/long and altitude, compute an ECIPoint for any given unix time."
    def __init__(self, lat, lon, alt_m):
        """Create an ECIPoint generator for the given position.
        
        * (lat,lon) in radians, because degress includes further decimal vs hms arbitrariness
        * alt above earth radius in meters
        """
        self.lat = lat
        self.lon = lon
        self.alt = alt_m/1e3 # Convert to km
        
    def lmst(self, t):
        # Get the local sideral mean time for unix time t, in radians
        
        gmst = GMST.from_unix(t)
        theta = gmst + self.lon # local mean sidereal time
        return theta
        
    def __repr__(self):
        return "(%0.4f,%0.4f)@%0.1fm" % (np.rad2deg(self.lat), np.rad2deg(self.lon), self.alt*1e3)
        
    def at(self, t):
        # Returns the ECIPoint for the given position at time t
        
        theta = self.lmst(t)
        
        # Correct for spheroid
        c = np.sqrt(1 + WGS84.f*(WGS84.f-2)*(np.sin(self.lat)**2))
        sq = c*(1-WGS84.f)**2
        
        # no idea why this is called achcp. 
        achcp = (WGS84.a*sq + self.alt) * np.cos(self.lat)
        
        pos_x = achcp*np.cos(theta)
        pos_y = achcp*np.sin(theta)
        pos_z = (WGS84.a*c + self.alt)*np.sin(self.lat)
        
        vel_x = WGS84.omega * -1*pos_y # derivative of sin and cos are handy here
        vel_y = WGS84.omega *    pos_x
        vel_z = 0 # please don't throw your telescope, or this gets super ugly.
        
        return ECIPoint((pos_x,pos_y,pos_z), (vel_x,vel_y,vel_z))
    
    
    def v_to(self, t, pt):
        # Get a vector in topocentric coordinates at the given ECIPoint at unix time t
        
        obs_pt = self.at(t)
        theta = self.lmst(t)
        
        v_range = obs_pt.range_to(pt)
        
        l_c,l_s = np.cos(self.lat), np.sin(self.lat) # we use these a lot below
        t_c,t_s = np.cos(theta), np.sin(theta)
        
        # The axes used here are slightly annoying to me: they're due south, due east, and up.
        # This is almost certainly done to keep some handedness constraints in the math, but,
        # dangit three-space, why are you so difficult?
        xform = np.array([
            [l_s*t_c, l_s*t_s, -l_c],
            [-t_s,    t_c,        0],
            [l_c*t_c, l_c*t_s,  l_s]]) # prettier if you swap y and z, but that's confusing
        
        vec_sez = xform.dot(v_range)
        return vec_sez
    
    def alt_az(self, t, pt):
        # Returns the elevation and azimuth of the given point in space at time t from here
        vec_sez = self.v_to(t,pt)
        distance = np.sqrt(np.sum(vec_sez**2))
        
        azimuth = np.arctan2(vec_sez[1], -1*vec_sez[0])
        elevation = np.arcsin(vec_sez[2]/distance)
        
        if azimuth < 0:
            azimuth += 2*np.pi

        return (elevation, azimuth)

Populating the interactive namespace from numpy and matplotlib


In [2]:
qth_deg = (37.728206,-122.407863) # QTH for AJ9BM
qth = ECIEarthPoints(np.deg2rad(qth_deg[0]), np.deg2rad(qth_deg[1]), 25)

Okay, let's get the ISS TLEs from Celestrak's [stations.txt file](https://www.celestrak.com/NORAD/elements/stations.txt).  The explanations of these are beyond the scope of our current project, thankfully.  Maybe another day.

Also, update gpredict using its built in TLE updater at the same time.  You can verify the TLEs are identical by checking `~/.config/Gpredict/satdata/25544.sat` (25544 is the SSN for ISS).  The ISS was literally overhead as I ran all this, so it was a bit rushed the first time through, but the TLEs matched.

In [3]:
iss_tles="""ISS (ZARYA)             
1 25544U 98067A   17330.99569027  .00003456  00000-0  59208-4 0  9997
2 25544  51.6427 306.6944 0004138 155.0450 324.2726 15.54239375 87077
"""

In [4]:
from sgp4.earth_gravity import wgs84
from sgp4.io import twoline2rv

In [5]:
iss_sat = twoline2rv(iss_tles.split("\n")[1], iss_tles.split("\n")[2], wgs84)

In [6]:
import time

In [7]:
class ECITLEPoints:
    """Generator for an orbit described by TLEs"""
    def __init__(self, tle1, tle2, gravity_model):
        self.tle1 = tle1
        self.tle2 = tle2
        self.wgs = gravity_model
        
        self.satellite = twoline2rv(tle1, tle2, gravity_model)
        
    def at(self, t):
        # Get the ECIPoint for the satellite at the given time
        tm = time.gmtime(t)
        pos,vel = self.satellite.propagate(tm.tm_year, tm.tm_mon, tm.tm_mday,
                                           tm.tm_hour, tm.tm_min, tm.tm_sec) # why. just why.
        return ECIPoint(pos,vel)

In [8]:
t = time.time()
tm = time.gmtime(t)
position, vel = iss_sat.propagate(tm.tm_year, tm.tm_mon, tm.tm_mday, tm.tm_hour, tm.tm_min, tm.tm_sec)
iss_eci = ECIPoint(position, vel)

map(np.rad2deg,qth.alt_az(t, iss_eci))

[-1.8739948717186834, 270.6455109864869]

In [40]:
iss = ECITLEPoints(iss_tles.split("\n")[1], iss_tles.split("\n")[2], wgs84)
t_now = time.time()
map(np.rad2deg, qth.alt_az(t_now, iss.at(t_now)))

[-39.939552862651702, 71.904095039144664]

In [41]:
t_now = 1511750469
expected = (67.51,241.21)
# expected range: 439km, range rate -2.593km/s, altitude 408km, vel 7.671km/s
map(np.rad2deg, qth.alt_az(t_now, iss.at(t_now)))

[66.137006708800129, 230.28788402030125]

In [11]:
iss.at(t_now).speed()

7.670494131021

In [12]:
qa = qth.at(t_now)
ia = iss.at(t_now)

In [13]:
np.dot(qa.range_to(ia), qa.range_vel(ia))/np.linalg.norm(qa.range_to(ia))

-2.8114646837966091

In [14]:
pos,vel = iss_sat.propagate(2003,3,23,0,3,22)
np.rad2deg(np.arctan2(pos[2], np.sqrt(pos[0]**2+pos[1]**2))),np.linalg.norm(pos)-WGS84.a

(11.343860389246361, 407.10165883067657)

In [15]:
iss2_sat = twoline2rv("1 25544U 98067A   03097.78853147  .00021906  00000-0  28403-3 0  8652", "2 25544  51.6361  13.7980 0004256  35.6671  59.2566 15.58778559250029", wgs84)

In [16]:
pos,vel = iss2_sat.propagate(2003,3,23,0,3,22)
np.rad2deg(np.arctan2(pos[2], np.sqrt(pos[0]**2+pos[1]**2))),np.linalg.norm(pos)-WGS84.a

(23.025274886482308, 388.73878691717346)

In [17]:
import pyorbital

In [18]:
from pyorbital.orbital import Orbital
import datetime

In [19]:
orb = Orbital("ISS", line1=iss_tles.split("\n")[1], line2=iss_tles.split("\n")[2])
t_now = time.time()
now = datetime.datetime.utcnow()

orb.get_position(now), orb.get_lonlatalt(now)

((array([ 0.81527566, -0.30463416,  0.60999847]),
  array([-0.0062417 ,  0.06070707,  0.03849922])),
 (-150.01445129254606, 35.196049334544703, 407.70534505137545))

In [20]:
np.linalg.norm(iss.at(t_now).pos)-WGS84.a

400.64904879162623

In [21]:
orb.get_observer_look(datetime(, np.deg2rad(qth_deg[0]), np.deg2rad(qth_deg[1]), 0)

SyntaxError: invalid syntax (<ipython-input-21-88bc6c93b922>, line 1)

In [22]:
iss.at(t_now).pos, orb.get_position(now)[0]*WGS84.a

(array([ 5200.11670661, -1944.67445609,  3889.59738184]),
 array([ 5199.93985356, -1942.99840505,  3890.65379783]))

In [23]:
datetime.tzinfo

In [24]:
dir(datetime)

['MAXYEAR',
 'MINYEAR',
 '__doc__',
 '__name__',
 '__package__',
 'date',
 'datetime',
 'datetime_CAPI',
 'time',
 'timedelta',
 'tzinfo']

In [138]:
dir(tzinfo)

['__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'dst',
 'fromutc',
 'tzname',
 'utcoffset']

In [137]:
orb.get_observer_look(datetime(2017,11,27,2,40,50, 0, tzinfo.), 37.7282, -122.4070, 25)

TypeError: descriptor 'tzname' requires a 'datetime.tzinfo' object but received a 'str'

In [25]:
import ephem

In [47]:
iss = ephem.readtle(*(iss_tles.split("\n")[:3]))

In [48]:
iss.compute('2017-11-27 02:40:50')
iss.elevation

407794.5625

In [49]:
ephem.delta_t()

69.34734456399744

In [59]:
qth = ephem.Observer()
qth.lat=np.deg2rad(qth_deg[0])
qth.lon=np.deg2rad(qth_deg[1])
qth.elevation=25

qth.date = datetime.datetime.utcnow()

iss.compute(qth)
np.rad2deg(iss.alt), np.rad2deg(iss.az)

(-51.201164150639514, 77.117405125998701)

In [ ]:
iss.